In [ ]:
!pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
from konlpy.tag import Mecab
import pandas as pd
from gensim.models.word2vec import Word2Vec as wv
from tqdm import tqdm as tq
from google.colab import drive
import numpy as np
import os
from keras.utils import np_utils
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/Processing/lyrics_ko.csv', index_col=0)
data_train = data_train.reset_index(drop=True)
lyr = data_train['lyr_kor']
lyr

0                                             내 시간은 너에게 ...
1             영원한      전부 태워     전부 불태워 혼자 있는 날 안아줘   난 ...
2         너의 창을 향해 내린 햇살의 의미는 바로 나 바로 나 너의 두 뺨 위로 떨어진 눈물...
3         노란 불빛 가로등 하나 그대 집 앞 골목길 숨가쁜 오르막 마냥 좋았던 감나무 한그루...
4         목련이 필 때 따라 웃던 사람 목련이 지면 음 눈물짓는 그댈 알게 된 날에 목련의 ...
                                ...                        
160872    지긋지긋한 일상에 머린 지끈지끈해 하는 일 하나같이 매일 삐끗삐끗대 입맛은 줄어 짜...
160873    지금은 팔아버린  겹겹이 쌓인 옷 그사이 사이 고이 모셔 환전직전의 돈봉투 입김을 ...
160874    여전히 말하지      원이 전부였지만 내게도 꿈이 서른 살이 되 직전까지 손을 벌...
160875    너 없이 나 혼자 있을 때면 항상 난 니가 생각나 너 없인 하루조차 못 버티는 나잖...
160876    너 없이 나 혼자 있을 때면 항상 난 니가 생각나 너 없인 하루조차 못 버티는 나잖...
Name: lyr_kor, Length: 160877, dtype: object

In [ ]:
mecab = Mecab()

In [ ]:
def word2num(list_2d):
    w2n_dic = dict()  # word가 key이고 index가 value인 dict
    n2w_dic = dict()  # index가 key이고 word가 value인 dict. 나중에 번호에서 단어로 쉽게 바꾸기 위해.
    idx = 1
    num_list = [[] for _ in range(len(list_2d))]   # 숫자에 매핑된 글의 리스트
    for k,i in enumerate(list_2d):
        if not i:
            continue
        elif isinstance(i, str): 
             # 내용이 단어 하나로 이루어진 경우, for loop으로 ['단어']가 '단'과 '어'로 나뉘지 않게 한다.
            if w2n_dic.get(i) is None:
                w2n_dic[i] = idx
                n2w_dic[idx] = i
                idx += 1
            num_list[k] = [dic[i]]
        else:
            for j in i:
                if w2n_dic.get(j) is None:
                    w2n_dic[j] = idx
                    n2w_dic[idx] = j
                    idx += 1
                num_list[k].append(w2n_dic[j])
    return num_list, w2n_dic, n2w_dic

In [ ]:
lyr_split = []
for l in tq(lyr):
  lyr_split.append(mecab.pos(l))

100%|██████████| 160877/160877 [03:14<00:00, 829.00it/s]


In [ ]:
stopword=[]
for i in tq(lyr_split):
  for word, tag in i:
    if tag in 'SL' or tag in 'SF' or tag in 'SP' or tag in 'SY' or tag in 'SE' or tag in 'SS' or tag in 'SE' or tag in 'SO' or tag in 'SW' or tag in 'UN' or tag in 'OH'or tag in 'ON'or tag in 'OL'or tag in 'IC':
      stopword.append(word)

100%|██████████| 160877/160877 [00:19<00:00, 8132.16it/s]


In [ ]:
stopword_set = set(stopword)
stopword_list = list(stopword_set)
stopword = stopword_list
stopword

In [ ]:
tagg=[]
for i in tq(lyr):
  tag = mecab.morphs(i)
  tag = [word for word in tag if not word in stopword]
  tagg.append(tag)

num, w2n_dic, n2w_dic = word2num(tagg)
tagg

 86%|████████▌ | 138137/160877 [1:08:47<10:10:11,  1.61s/it]

In [ ]:
model_mecab = wv(tagg,min_count =1 ,sg=0, batch_words=1000, window = 10)

In [ ]:
model_mecab.wv.most_similar('슬픔')

In [ ]:
emotion_list=['즐거움','슬픔','분노','사랑','외로움', '불안']

In [ ]:
lyr_simil = []
for i in range(len(emotion_list)):
  emotion_simil=[]
  for word in model_mecab.wv.most_similar(emotion_list[i], topn=20):
    if word[1]>=0.5:
      emotion_simil.append(word)
  lyr_simil.append(emotion_simil)

In [ ]:
emotion_result = []
for z in tq(range(len(tagg))):
    lyric_emotion = ['즐거움','슬픔','분노','사랑','외로움', '불안']
    data_test = tagg[z]
    # 계산용 Dictionary 구성
    data_dic = {lyric_emotion[0]:0,lyric_emotion[1]:0,lyric_emotion[2]:0,
               lyric_emotion[3]:0,lyric_emotion[4]:0,lyric_emotion[5]:0}

    for k in range(len(lyr_simil)):
        for i in lyr_simil[k]:
            for j in data_test:
                if i[0] == j:
                    data_dic[lyric_emotion[k]] = data_dic[lyric_emotion[k]]+i[1]
    def whatIsBest(x):
        return data_dic[x]

    best_emotion = max(data_dic.keys(), key = whatIsBest)
    # print(best_emotion, data_dic[best_emotion])
    
    if data_dic[best_emotion] != 0:
        emotion_result.append(best_emotion)
        # 연관 단어가 있어서 점수가 나오면 가장 높은 감정을 리스트에 담음
    else:
        emotion_result.append('null') 
        # 연관 단어가 없으면 0으로 나옴. 리스트 입력 순서를 맞춰야 하기에 'null' 추가

In [ ]:
data_result = lyr
result_list = emotion_result

In [ ]:
result_pd = pd.Series(result_list)
data_label = pd.concat([data_result, result_pd], axis = 1)
data_label

In [ ]:
emotion_label = {'즐거움':0,'슬픔':1,'분노':2,'사랑':3,'외로움':4,'불안':5}
#['즐거움','슬픔','불안','사랑','분노','외로움']
data_label["emo_num"]=data_label.iloc[:,1].map(emotion_label)
data_label

In [ ]:
data_educate = data_label.drop(data_label[data_label.iloc[:,1] == 'null'].index, axis= 0)
data_educate = data_educate.dropna(axis=0)
data_educate

In [ ]:
data_educate.to_csv('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/Processing/educate_data.csv', index=False, encoding='utf-8')

In [ ]:
data_educate.columns=(1,2,'labels')
data_educate_result = data_educate.drop(2,axis=1)
data_educate_result

In [ ]:
X = data_educate_result[1]
y = data_educate_result['labels']

In [ ]:
X.shape, y.shape

In [ ]:
X_ex=[]
for i in tq(X):
  tag = mecab.morphs(i)
  tag = [word for word in tag if not word in stopword]
  X_ex.append(tag)

In [ ]:
data_educate_result['tokenize']=X_ex
data_educate_result

In [ ]:
data_educate_result.to_csv('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/Processing/data_edu_tokenize.csv', index=False, encoding='utf-8')

In [ ]:
num_list, w2n_dic, n2w_dic = word2num(X_ex)

In [ ]:
def divide_data(x, y, train_prop=0.8):
    x = np.array(x)
    y = np.array(y)
    tmp = np.random.permutation(np.arange(len(x)))
    x_tr = x[tmp][:round(train_prop * len(x))]
    y_tr = y[tmp][:round(train_prop * len(x))]
    x_te = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_te = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return x_tr, x_te, y_tr, y_te

In [ ]:
x_tr, x_te, y_tr, y_te = divide_data(num_list, y)

In [ ]:
x_tr.shape, x_te.shape, y_tr.shape, y_te.shape

In [ ]:
# 학습 데이터 크기 일정하게 지정
x_tr = sequence.pad_sequences(x_tr, maxlen=300)
x_te = sequence.pad_sequences(x_te, maxlen=300)

# 원핫 인코딩
y_tr = np_utils.to_categorical(y_tr) # one_hot으로 변형
y_te = np_utils.to_categorical(y_te) # 3 -> [0,0,0,1,0, ..]

In [ ]:
max_x = []
for i in x_tr:
  max_x.append(max(i))
max(max_x)

In [ ]:
# x_tr,x_te,y_tr,y_te
# words_num은 총 단어의 종류. +1을 해준 이유는 단어 수가 적은 글의 경우 빈 칸에 0이 있기 때문에.
model = Sequential()
model.add(Embedding(69000, len(x_tr[0])))  # 사용된 단어 수 & input 하나 당 size
model.add(LSTM(len(x_tr[0])))
model.add(Dense(len(y_tr[0]), activation='softmax'))  # 카테고리 수 (원핫해뒀기때문에 이대로)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
es_callback = EarlyStopping(monitor = 'val_loss', patience=30)

In [ ]:
# 모델을 생성할 폴더
MODEL_DIR = '/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/Processing/model/'

# 폴더가 없다면 폴더 생성
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
# 저장할 파일명의 정규식을 설정(epoch수 - 손실값 을 파일명으로 설정)
modelpath = MODEL_DIR + 'final.h5'

# monitor = val_loss : 모니터링할 값(val_loss : 테스트오차, val_ass : 테스트 정확도)
# save_best_only : 이전에 저장한 모델보다 나아질경우 저장
checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_accuracy',verbose=1, save_best_only=True)

In [ ]:
history = model.fit(x_tr, y_tr, batch_size=100, epochs=200, callbacks=[es_callback, checkpointer],  validation_data=(x_te,y_te))

In [ ]:
modelpath = MODEL_DIR + 'final.h5'
load_model = load_model(modelpath)
load_model.summary()

In [ ]:
load_model.evaluate(x_te, y_te)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()